In [1]:
from data import get_data, get_processed_data
from train_test import train, output_to_accu, test
from helpers import plot_cm, count_parameters
from models import MLP, ConvNet, LSTM
from sklearn.model_selection import train_test_split
import torch
import numpy as np

import shap

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
_, target_prices, _, features = get_data()

input_period = 4
input_period_reg = 10
training_window = 5

X, _, y, _ = get_processed_data(features, target_prices, input_period, input_period_reg, training_window)

In [13]:
train_indices, test_indices, _, _ = train_test_split(range(len(y)), y, test_size=0.2, shuffle=False)
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]
print('Number of train sample :', len(X_train))
print('Number of test sample :', len(X_test))

X_mean = X_train.mean(dim=[0, 1], keepdim=True)
X_std = X_train.std(dim=[0, 1], keepdim=True)
X_train = X_train.sub_(X_mean).div_(X_std)
X_test = X_test.sub_(X_mean).div_(X_std)

Number of train sample : 1046
Number of test sample : 262


In [14]:
eta = 1e-3
weight_decay = 1e-4
dropout = 0.2
batch_size = 30
nb_epochs = 20

verbose = 3

model_name = 'MLP'

dim1, dim2 = X.size(1), X.size(2)

if model_name == 'MLP':
    model = MLP(dim1, dim2, pdrop=dropout)

elif model_name == 'ConvNet':
    model = ConvNet(dim1, dim2, pdrop=dropout)

elif model_name == 'LSTM':
    model = LSTM(input_size=dim2, output_size=y.size(1), device=device, pdrop=dropout)
    
train(model, X_train, y_train, nb_epochs=nb_epochs, device=device, X_test=X_test, y_test=y_test, 
      batch_size=batch_size, eta=eta, weight_decay=weight_decay, verbose=verbose)

100%|██████████| 20/20 [00:01<00:00, 13.33it/s]


In [11]:
explainer = shap.DeepExplainer(model, X_test.narrow(0, 0, 100))
shap_values = explainer.shap_values(X_test.narrow(0, 100, 1))

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
